In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings("ignore")

sns.set_style("whitegrid")

In [2]:
df = pd.read_csv("cleaned_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40450 entries, 0 to 40449
Data columns (total 11 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   Unnamed: 0                                         40450 non-null  int64 
 1   Date                                               40450 non-null  int64 
 2   Vehicle Category                                   40450 non-null  object
 3   GVWR Class                                         40450 non-null  object
 4   Fuel Type                                          40450 non-null  object
 5   Model Year                                         40450 non-null  int64 
 6   Fuel Technology                                    40450 non-null  object
 7   Electric Mile Range                                40450 non-null  object
 8   Number of Vehicles Registered at the Same Address  40450 non-null  object
 9   Region           

In [4]:
df = df.drop(columns="Region", axis = 1)

In [ ]:
df["Vehicle Category"] = 